<a href="https://colab.research.google.com/github/Athanas7a/AI-Crash-Course/blob/master/Create_AI_Generated_Images_with_Dalle3_in_bulk_via_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions



1. File > Save a copy in Drive
2. Upload a CSV with 3 headers: `prompt,num_images,keyword` (name it input.csv)
3. Add your API_KEY (from openAI) below in the code
4. In the menu click `Runtime > Run all` & wait
5. Look in the `generated_images` folder to download one by one
6. Refresh the side area to download the .zip

---


📺 Video Tutorial: https://youtu.be/1nyDvfCoqLs


---

## Questions?

- [Join Discord](https://serp.ly/@serp/discord)
- [SERP](https://serp.co)
- [SERP AI](https://serp.ai)
- [DevinSchumacher.com](https://devinschumacher.com)
- [YouTube Tutorials](https://serp.ly/@devin/youtube)

In [ ]:
# Add your OpenAI API key here

API_KEY = "sk-xxx"


# Code

👇 Click this to "Run all"

In [ ]:
# CSV

csv_file_path = "/content/input.csv"

!pip install openai tiktoken cohere

import csv
from openai import OpenAI
import requests
from IPython.display import display, Image
from io import BytesIO
import os


def read_prompts_from_csv(file_path):
    prompts = []
    with open(file_path, newline="", encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            prompts.append(
                {
                    "prompt": row["prompt"],
                    "num_images": int(row["num_images"]),
                    "keyword": row["keyword"],
                }
            )
    return prompts


def generate_dalle_images(prompts, save_path):
    headers = {"Authorization": f"Bearer {API_KEY}"}

    # Create the directory if it does not exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for prompt_details in prompts:
        prompt = prompt_details["prompt"]
        num_images = prompt_details["num_images"]
        keyword = prompt_details["keyword"]

        for i in range(num_images):
            data = {
                "model": "dall-e-3",
                "prompt": prompt,
                "n": 1,
                "size": "1792x1024",
                "quality": "hd",
            }

            response = requests.post(
                "https://api.openai.com/v1/images/generations",
                headers=headers,
                json=data,
            )

            if response.status_code == 200:
                images = response.json()["data"]
                for img in images:
                    image_response = requests.get(img["url"])
                    if image_response.status_code == 200:
                        image = Image(BytesIO(image_response.content))
                        display(image)

                        # Save the image with a formatted name
                        image_file_name = f"{keyword.replace(' ', '_')}_{i+1:02d}.png"
                        image_file_path = os.path.join(save_path, image_file_name)
                        with open(image_file_path, "wb") as f:
                            f.write(image_response.content)
                        print(f"Image saved to {image_file_path}")
                    else:
                        print("Error loading image:", image_response.status_code)
            else:
                print("Error:", response.status_code, response.text)

In [ ]:
# Example usage with CSV

prompts_from_csv = read_prompts_from_csv(csv_file_path)

generate_dalle_images(prompts_from_csv, "./generated_images")

!zip -r /content/generated_images.zip /content/generated_images/